In [66]:
import sqlite3
import pandas as pd
import numpy as np
import re
from datetime import date
from datetime import datetime

df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')

dfPI = pd.read_sql_query("SELECT * FROM personal_info", df)


#Wrangling personal_info
    #Drop Unnamed:0 and lastName, I delete lastname column since we already have the full name and surname in rank_info
    
dfPI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfPI.drop(['lastName'], axis='columns', inplace=True)


    #Wrangling gender column
    
dfPI.gender.replace('[M]+$', 'Male', regex=True, inplace=True)
dfPI.gender.replace('[F]+$', 'Female', regex=True, inplace=True)
dfPI['gender'] = dfPI['gender'].astype('object').astype('str')
#print(dfPI['gender'].value_counts())
   
    #Wrangling age column
        #Correct info in age column: delete "years old" and change data types
        
dfPI.age.replace('[years old]+$', '', regex=True, inplace=True)   
dfPI['age'] = dfPI['age'].fillna(0)   #replace NaN values with 0.0
dfPI['age'] = dfPI['age'].astype('float').astype('int') 
#print(dfPI['age'].value_counts())    
#print(dfPI['age'].dtype)  

#Checking Types: 

#print(dfPI['id'].dtype) 
#print(dfPI['age'].dtype)

    #Datetime - preparing 

now = datetime.now()
now.year

    #Checking Outliers: 

stats = dfPI.describe().T
stats['IQR'] = stats['75%'] - stats['25%']
stats

outliers = pd.DataFrame(columns=dfPI.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = dfPI[(dfPI[col] < lower) | 
                   (dfPI[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
    'sort=False'
     
dfPI['new_age'] = now.year - outliers['age']

dfPI.new_age.fillna(dfPI.age, inplace=True)

dfPI['age'] = dfPI['new_age']
dfPI.drop(['new_age'], axis='columns', inplace=True)

#Checking Duplicates

dfPI = dfPI.drop_duplicates()

#RESET INDEX

dfPI = dfPI.reset_index(drop=True)

#Exporting the clean dataset to the data/processed folder

dfPI.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/dfPersonalInfoCLEAN.csv', index=False)
dfPI

,id,age,gender,country,image
0,4656,54,Male,None,https://specials-images.forbesimg.com/imageser...
1,1874,62,None,United States,https://specials-images.forbesimg.com/imageser...
2,6703,87,Male,United States,https://specials-images.forbesimg.com/imageser...
3,2637,69,None,None,https://specials-images.forbesimg.com/imageser...
4,8242,35,Male,None,https://specials-images.forbesimg.com/imageser...
...,...,...,...,...,...
2203,1711,51,None,China,https://specials-images.forbesimg.com/imageser...
2204,3538,55,Male,None,https://specials-images.forbesimg.com/imageser...
2205,7797,51,None,China,https://specials-images.forbesimg.com/imageser...
2206,3123,52,Female,None,https://specials-images.forbesimg.com/imageser...
